In [1]:
#import scipy
import numpy as np
import logging
from ridge_utils import mult_diag, counter
import random
import itertools as itools

In [2]:

zs = lambda v: (v-v.mean(0))/v.std(0) ## z-score function

ridge_logger = logging.getLogger("ridge_corr")

def ridge(stim, resp, alpha, singcutoff=1e-10, normalpha=False, logger=ridge_logger):
 
    try:
        U,S,Vh = np.linalg.svd(stim, full_matrices=False)
    except np.linalg.LinAlgError:
        logger.info("NORMAL SVD FAILED, trying more robust dgesvd..")
        from text.regression.svd_dgesvd import svd_dgesvd
        U,S,Vh = svd_dgesvd(stim, full_matrices=False)

    UR = np.dot(U.T, np.nan_to_num(resp))
    
    # Expand alpha to a collection if it's just a single value
    if isinstance(alpha, (float,int)):
        alpha = np.ones(resp.shape[1]) * alpha
    
    # Normalize alpha by the LSV norm
    norm = S[0]
    if normalpha:
        nalphas = alpha * norm
    else:
        nalphas = alpha

    # Compute weights for each alpha
    ualphas = np.unique(nalphas)
    wt = np.zeros((stim.shape[1], resp.shape[1]))
    for ua in ualphas:
        selvox = np.nonzero(nalphas==ua)[0]
        #awt = reduce(np.dot, [Vh.T, np.diag(S/(S**2+ua**2)), UR[:,selvox]])
        awt = Vh.T.dot(np.diag(S/(S**2+ua**2))).dot(UR[:,selvox])
        wt[:,selvox] = awt

    return wt


In [4]:
import numpy as np
import npp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split



fMRI_Data= np.load('fMRI_Data_subject3.npy', allow_pickle=True)
Movie_Data= np.load('Movie_Data_subject3.npy', allow_pickle=True)


print (Movie_Data.shape, fMRI_Data.shape)

fMRI_Data=fMRI_Data [0:20000,:]
Movie_Data=fMRI_Data [0:20000,:]


print (Movie_Data.shape, fMRI_Data.shape)


x_train,x_test, y_train, y_test = train_test_split(Movie_Data, fMRI_Data , test_size=0.1)
print(x_train.shape,x_test.shape, y_train.shape, y_test.shape)





#from ridge import ridge
beta_ridge = ridge (x_train,y_train, alpha=15.)

wt=beta_ridge





print(wt.shape)

pred_test = x_test.dot(wt)

Ridge_correlations = npp.mcorr(y_test, pred_test)

print(Ridge_correlations.shape)



np.save('1Ridge_correlations.npy', Ridge_correlations)

np.save('wt.npy', wt)


(43999, 4096) (43999, 444)
(20000, 444) (20000, 444)
(18000, 444) (2000, 444) (18000, 444) (2000, 444)
(444, 444)
(444,)
